In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


In [0]:
import tensorflow as tf
import tensorflow.contrib.keras.api.keras.backend as K

from tensorflow.contrib.keras.api.keras.optimizers import Adam
from tensorflow.contrib.keras.api.keras.models import Model, Sequential
from tensorflow.contrib.keras.api.keras.layers import Input , Activation
from tensorflow.contrib.keras.api.keras.layers import Conv2D, Reshape
from tensorflow.contrib.keras.api.keras.layers import Dropout,BatchNormalization
from tensorflow.contrib.keras.api.keras.layers import concatenate

import numpy as np
import os
import random

from scipy.misc import imsave
import matplotlib.pyplot as plt

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
""" 
// Step1-1 train/validation/test 데이터 로드. // 

img_L: 0 ~ 255
img_R: 0 ~ 255
disp_L: -1 ~ 18.37 (px)
disp_R: -1 ~ 18.37 (px)
"""

img_L=1/255*np.load('drive/colab/data_stereo/img_L.npy')
img_R=1/255*np.load('drive/colab/data_stereo/img_R.npy')
disp_L=np.load('drive/colab/data_stereo/disp_L.npy')
disp_R=np.load('drive/colab/data_stereo/disp_R.npy')

img_L_val=1/255*np.load('drive/colab/data_stereo/img_L_val.npy')[np.newaxis,:,:,:]
img_R_val=1/255*np.load('drive/colab/data_stereo/img_R_val.npy')[np.newaxis,:,:,:]
disp_L_val=np.load('drive/colab/data_stereo/disp_L_val.npy')
disp_R_val=np.load('drive/colab/data_stereo/disp_R_val.npy')

img_L_test=1/255*np.load('drive/colab/data_stereo/img_L_test.npy')
img_R_test=1/255*np.load('drive/colab/data_stereo/img_R_test.npy')
img_L_test=np.transpose(img_L_test,[3,0,1,2])
img_R_test=np.transpose(img_R_test,[3,0,1,2])
# 데이터 shape 받아오기 (image_h, image_w, 3, )
image_h=img_L.shape[0] # 116
image_w=img_L.shape[1] # 170    

In [0]:
print('img_L shape: %sx%sx%sx%s' % img_L.shape)
print('img_R shape: %sx%sx%sx%s' % img_R.shape)
print('disp_L shape: %sx%sx%s' % disp_L.shape)
print('disp_R shape: %sx%sx%s' % disp_R.shape)

plt.subplot(131); plt.grid(False); plt.title('img_L')
plt.imshow(img_L[:,:,:,0])
plt.subplot(132); plt.grid(False); plt.title('img_R')
plt.imshow(img_R[:,:,:,0])
plt.subplot(133); plt.grid(False); plt.title('disp_L')
plt.imshow(disp_L[:,:,0])

In [0]:
""" 
// Step1-2 기타 setting  // 

conv_depth: Conv2D 레이어 개수
filt_num: Conv2D 필터 개수

patch_h, patch_w: 트레이닝할 때 사용할 patch 크기
batch_size, learning_rate
one_step, epoches
"""

network_name="stereo_basic"
save_bool = False

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"  

conv_depth=8
filt_num=128

batch_size=32
learning_rate=1e-4
decay_rate=1e-6

patch_h=1
patch_w=33

one_step=3000 
epoches=20
iter00=0  


In [0]:
def mse_valid(y_true, y_pred):
    valid_mask= tf.cast(tf.less(0.0,y_true),tf.float32)
    return K.mean(valid_mask*K.square(y_pred - y_true), axis=-1)

In [0]:
def network_stereo(patch_h,patch_w,conv_depth,filt_num):
    """
    네트워크 설계
    < Layer 사용 예시 >
    output=concatenate([input1,input2])
    input1=Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', 
                  dilation_rate=(1, 1))(input1)
    input1=Activation('relu')(input1)
    input1=BatchNormalization()(input1)
    
    """
    
    ''' Define Input_L, input_R  ''' 
    input_L = Input(shape=(patch_h,patch_w,3), name='input_L')
    input_R= Input(shape=(patch_h,patch_w,3), name='input_R')
    
    """
    write your code
    """
    
    model = Model(inputs = [input_L,input_R], outputs = [im_stack])
    
    return model  

In [0]:
""" 
// Step2 네트워크 설계 //

training에서 사용할 모델 정의 """ 
model_train=network_stereo(patch_h,patch_w,conv_depth,filt_num)
opt = Adam(lr=learning_rate,decay=decay_rate)
model_train.compile(optimizer=opt, loss='mse')
model_train.summary() 

""" val/test에서 사용할 모델 정의  """

model_val=network_stereo(image_h,image_w,conv_depth,filt_num)
model_val.compile(optimizer=opt, loss='mse')


""" input/output size가 다를때 크기 맞추기 위해 crop  """
sliced_half=(patch_w-model_train.output_shape[-1])//2
disp_L_val=disp_L_val[:,sliced_half:-sliced_half]

In [0]:
""" 
// Step3 생성함수 //   """ 

""" 
data 생성함수 정의 (fit_generator에서 사용할 것임)
FCN 이므로 patch-wise training으로 학습하는 것이 학습속도 빠름.
patch 크기: (patch_h, patch_w)
이미지 랜덤으로 잘라서 batch_img_L: (batch_size,patch_h, patch_w) 만든다.

""" 
def dataGenerator(img_L,img_R,disp_L,disp_R, batch_size, patch_h, patch_w, sliced_half): 
    while 1:
        """
        실시간으로 트레이닝 데이터 생성하는 함수 (fit_generator에서 사용할 예정)
        이미지 사이즈에 맞춰 랜덤으로 위치를 바꿔가며 batch_size만큼 생성한다.
        Data augmentation(color/flip 등)도 해보기.
        
        <Input>등
        img_L : (116, 170, 3, 160) 160장의 Left image
        img_R : (116, 170, 3, 160) 160장의 Right image
        disp_L : (116, 170, 3)     160장의 disparity image
        batch_size,                배치 크기
        patch_h, patch_w           crop할 크기(패치 크기)
        sliced_half                zero-padding 없을 때, 이미지가 작아지는 값의 1/2
        
        <Output> 
        batch_img_L: (batch_size, patch_h, patch_w, 3)
        batch_img_R: (batch_size, patch_h, patch_w, 3)
        batch_disp_L: (batch_size, patch_h, int(patch_w-2*sliced_half))
        """
        
        batch_img_L = np.zeros((batch_size, patch_h, patch_w, 3))
        batch_img_R = np.zeros((batch_size, patch_h, patch_w, 3))
        batch_disp_L= np.zeros((batch_size, patch_h, int(patch_w-2*sliced_half)))
        
        for _ in range(batch_size):
            image_id=int(img_L.shape[-1]*random.random())
            
            """
            write your code
            
            batch_img_L[_,] ?
            batch_img_R[_,] ?
            batch_disp_L[_,] ?  
            
            """           

        yield([batch_img_L,batch_img_R],batch_disp_L) 

my_generator = dataGenerator(img_L,img_R,disp_L,disp_R, batch_size, patch_h, patch_w, sliced_half)


In [0]:
""" 
// Step4 학습 시작 //   

총 epoches만큼 트레이닝"""  
for i in range((epoches)):


    """ Training:
        3 epoch씩 학습하고, 
        1 epoch는 3000 """
    model_train.fit_generator(my_generator, steps_per_epoch = one_step, 
                        epochs = iter00+3,  initial_epoch=iter00,
                        verbose=1, workers=1)
    iter00=iter00+3



    """ Validation: 
        model_train에서 학습한 weight을 
        model_val에 대입하여 error 확인한다.
    """
    weight_tmp1=model_train.get_weights() 
    model_val.set_weights(weight_tmp1)

    disp_L_predict=model_val.predict_on_batch([img_L_val, img_R_val])
    disp_L_predict=np.squeeze(disp_L_predict)
    loss_mae=np.sum((disp_L_val>0)*np.abs(disp_L_predict-disp_L_val))/image_h/(image_w-2*sliced_half) 
    print("loss = %.03f" % (loss_mae))



    """ Test: 

    """                

    disp_L_test_predict=model_val.predict([img_L_test, img_R_test],batch_size=1)
    disp_L_test_predict=np.squeeze(disp_L_test_predict)   


    """ 
    결과 저장
    ( 이미지->jpg, weights->hdf5 )  
    """  
    test_id1=2
    plt.subplot(231); plt.axis('off'); plt.title('image_L')
    plt.imshow(img_L_test[test_id1,])
    plt.subplot(232); plt.axis('off'); plt.title('image_R')
    plt.imshow(img_R_test[test_id1,])
    plt.subplot(233); plt.axis('off'); plt.title('disparity')
    plt.imshow(disp_L_test_predict[test_id1,],cmap='jet')
    test_id2=7
    plt.subplot(234); plt.axis('off'); plt.title('image_L')
    plt.imshow(img_L_test[test_id2,])
    plt.subplot(235); plt.axis('off'); plt.title('image_R')
    plt.imshow(img_R_test[test_id2,])
    plt.subplot(236); plt.axis('off'); plt.title('disparity')
    plt.imshow(disp_L_test_predict[test_id2,],cmap='jet')
    plt.savefig("drive/colab/save/%s_iter%04d_%.03f.jpg" % (network_name,iter00,loss_mae))  
    
    save_path="drive/colab/save/%s_iter%04d_%.03f.hdf5" % (network_name,iter00,loss_mae)
    if(save_bool==True):
        model_train.save_weights(save_path)
    print(save_path)